# graphblas.convert_layout

This example will go over how to use the `--graphblas-lower` pass from `graphblas-opt` to lower the `graphblas.convert_layout` op.

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [1]:
import mlir_graphblas
import mlir_graphblas.sparse_utils
import numpy as np

engine = mlir_graphblas.MlirJitEngine()

Here are the passes we'll use.

In [2]:
passes = [
    "--graphblas-lower",
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]

The GraphBLAS dialect's ops operate on and usually return sparse matrices. To analyze the results of the ops, it'd be nice to have some functions that'll convert those sparse matrices into dense matrices. The MLIR code below does that. 

We'll have 2 functions, `csr_densify4x4` and `csc_densify4x4`, which will return dense matrices from 4x4 sparse matrices in CSR and CSC formats, respectively. 

In [3]:
mlir_text = """
#trait_densify_csr = {
  indexing_maps = [
    affine_map<(i,j) -> (i,j)>,
    affine_map<(i,j) -> (i,j)>
  ],
  iterator_types = ["parallel", "parallel"]
}

#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csr_densify4x4(%argA: tensor<4x4xf64, #CSR64>) -> tensor<4x4xf64> {
  %output_storage = constant dense<0.0> : tensor<4x4xf64>
  %0 = linalg.generic #trait_densify_csr
    ins(%argA: tensor<4x4xf64, #CSR64>)
    outs(%output_storage: tensor<4x4xf64>) {
      ^bb(%A: f64, %x: f64):
        linalg.yield %A : f64
    } -> tensor<4x4xf64>
  return %0 : tensor<4x4xf64>
}

#trait_densify_csc = {
  indexing_maps = [
    affine_map<(i,j) -> (j,i)>,
    affine_map<(i,j) -> (i,j)>
  ],
  iterator_types = ["parallel", "parallel"]
}

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csc_densify4x4(%argA: tensor<4x4xf64, #CSC64>) -> tensor<4x4xf64> {
  %output_storage = constant dense<0.0> : tensor<4x4xf64>
  %0 = linalg.generic #trait_densify_csc
    ins(%argA: tensor<4x4xf64, #CSC64>)
    outs(%output_storage: tensor<4x4xf64>) {
      ^bb(%A: f64, %x: f64):
        linalg.yield %A : f64
    } -> tensor<4x4xf64>
  return %0 : tensor<4x4xf64>
}
"""

Let's compile our MLIR code. 

In [4]:
engine.add(mlir_text, passes)

['csr_densify4x4', 'csc_densify4x4']

## Overview of graphblas.convert_layout

Here, we'll show how to use the `graphblas.convert_layout` op. 

This op takes 1 sparse matrix in CSR or CSC format and creates a new sparse matrix of the desired format.

We'll give several examples below of how this will work.

First, we'll define an example input CSR matrix.

In [5]:
indices = np.array(
    [
        [0, 0],
        [1, 2],
    ],
    dtype=np.uint64,
)
values = np.array([1.1, 2.2], dtype=np.float64)
sizes = np.array([4, 4], dtype=np.uint64)
sparsity = np.array([False, True], dtype=np.bool8)

csr_matrix = mlir_graphblas.sparse_utils.MLIRSparseTensor(indices, values, sizes, sparsity)

In [6]:
engine.csr_densify4x4(csr_matrix)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

## graphblas.convert_layout (CSR->CSC)

Let's convert this matrix to CSC format.

In [7]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csr_to_csc(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}
"""

In [8]:
engine.add(mlir_text, passes)

['csr_to_csc']

In [9]:
csc_matrix = engine.csr_to_csc(csr_matrix)

In [10]:
engine.csc_densify4x4(csc_matrix)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

The result looks sane. Let's verify that it's exactly the same as the CSR matrix when both are converted to dense matrices.

In [11]:
np.all(engine.csc_densify4x4(csc_matrix) == engine.csr_densify4x4(csr_matrix))

True

## graphblas.convert_layout (CSC->CSR)

Let's convert the CSC matrix back to CSR format.

Let's first get rid of our original `csr_matrix` so we don't get correct results purely by accident.

In [12]:
del csr_matrix

Here's the MLIR code to convert from CSC to CSR. 

In [13]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csc_to_csr(%sparse_tensor: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSC64> to tensor<?x?xf64, #CSR64>
    return %answer : tensor<?x?xf64, #CSR64>
}
"""

In [14]:
engine.add(mlir_text, passes)

['csc_to_csr']

In [15]:
csr_matrix = engine.csc_to_csr(csc_matrix)

In [16]:
engine.csr_densify4x4(csr_matrix)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

The result looks sane. Let's verify that it's exactly the same as the CSC matrix when both are converted to dense matrices.

In [17]:
np.all(engine.csr_densify4x4(csr_matrix) == engine.csc_densify4x4(csc_matrix))

True

## graphblas.convert_layout (CSC->CSC, CSR->CSR)

For completeness, we'll show how to convert to and from the same exact layouts.

The MLIR code to do so is shown below.

In [18]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csc_to_csc(%sparse_tensor: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSC64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}

func @csr_to_csr(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSR64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>
    return %answer : tensor<?x?xf64, #CSR64>
}
"""

In [19]:
engine.add(mlir_text, passes)

['csc_to_csc', 'csr_to_csr']

Let's verify that converting to and from the same layout give correct results.

In [20]:
csc_result = engine.csc_to_csc(csr_matrix)
np.all(engine.csc_densify4x4(csr_matrix) == engine.csc_densify4x4(csc_result))

True

In [21]:
csr_result = engine.csr_to_csr(csr_matrix)
np.all(engine.csr_densify4x4(csr_matrix) == engine.csr_densify4x4(csr_result))

True